In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
'''sys.path.append('/home/wangs/rs/lwm/lib')'''
import ff
import pandas as pd
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import matplotlib as mpl
mpl.rc("font", family='Droid Sans Fallback', weight="bold")
import extend
# from scipy.stats import pearsonr
import matplotlib.pyplot as plt
# import mplfinance as mpf
import optuna as opt
from IPython.display import HTML
import warnings
import logging
warnings.filterwarnings('ignore')
opt.logging.set_verbosity(opt.logging.WARNING)

价量配合          
用成交量的放大或者是缩小，来给价格趋势投赞成票或是反对票           
选定影线差为价格因子去配合 Turn20 与 STR 因子，构建了效果优秀的 TPS 与 SPS 因子         
辅助因子1：日内收益率；构建方法为，今日收盘价减去今日开盘价的差除以昨日收盘价           
辅助因子2：是振荡幅度，振幅的构建方法为，今日最高价减去今日最低价的差除以昨日收盘价                
辅助因子3：影线差。叫做 PLUS , 英文是 the Premium between Lower and Upper Shadow, PLUS 的构建方法为上影线减去下影线后除以昨日收盘价。      $ e{PLS}_t=\frac{2 C_t-H_t-L_t}{C_{t-1 }   

In [ ]:
一. 要用到的三个因子Turn20、STR、PLUS
计算方式
Turn20：一个月换手率的均值
STR：一个月换手率的标准差
PLUS：见上
二. 纯净化因子

In [150]:
# 得到因子Turn、PLUS
# 这里fre是数据频率，n1为换手率周期,n2为因子计算周期
def get_factor1(para):
    name,fre,n1 = para
    data = ff.read_ba(name,k = f'{fre}H')
    data['Turn'] = data.v/data.v.rolling(n1,min_periods=1).mean()
    # data['Turn_n'] = data.Turn.rolling(n2).mean() # 对应Turn20
    # data['STR'] = data.Turn.rolling(n2).std() # 对应STR
    data['PLUS'] = (2*data.c-data.h-data.l)/data.c.shift(1)
    return data[['Turn','PLUS']]
# fre = 4
# n1 = 3
# n2 = 6
# para_lst = [(name,fre,n1) for name in ff.symbols]
# turn_lst = []
# plus_lst = []
# with Pool(8) as p:
#     res_lst = list(tqdm(p.imap(get_factor1, para_lst), total=len(para_lst)))
#     for res in res_lst:
#         turn_lst.append(res['Turn'])
#         plus_lst.append(res['PLUS'])
# factor_dic = {ff.symbols[i] : res_lst[i] for i in range(len(para_lst))}

# Turn = pd.concat([df['Turn'] for df in factor_dic.values()], axis=1)
# Turn.columns = factor_dic.keys()
# PLUS = pd.concat([df['PLUS'] for df in factor_dic.values()], axis=1)
# PLUS.columns = factor_dic.keys()

In [151]:
def get_factor2(para):
    i,date_Turn,date_PLUS = para
    valid_indices = np.logical_and(~np.isnan(date_Turn), ~np.isnan(date_PLUS))
    date_Turn_valid = date_Turn[valid_indices]
    date_PLUS_valid = date_PLUS[valid_indices]

    PLUS_deTurn = np.full_like(date_Turn, np.nan)
    Turn_dePLUS = np.full_like(date_Turn, np.nan)
    # 防止所有位置均为空值的情况
    try:
        date_Turn_valid_c = sm.add_constant(date_Turn_valid)
        date_PLUS_valid_c = sm.add_constant(date_PLUS_valid)
        
        PLUS_deTurn_model = sm.OLS(date_PLUS_valid, date_Turn_valid_c)
        Turn_dePLUS_model = sm.OLS(date_Turn_valid, date_PLUS_valid_c)
        
        PLUS_deTurn_results = PLUS_deTurn_model.fit()
        Turn_dePLUS_results = Turn_dePLUS_model.fit()    
    
        PLUS_deTurn[valid_indices] = PLUS_deTurn_results.resid  
        Turn_dePLUS[valid_indices] = Turn_dePLUS_results.resid  
    except:
        pass    
    return PLUS_deTurn,Turn_dePLUS

In [156]:
fre = 4
n1 = 20
n2 = 20
get_factor1_para_lst = [(name,fre,n1) for name in ff.symbols]
turn_lst = []
plus_lst = []
with Pool(16) as p:
    get_factor1_res_lst = list(tqdm(p.imap(get_factor1, get_factor1_para_lst), total=len(get_factor1_para_lst)))  
Turn = pd.DataFrame([res['Turn'] for res in get_factor1_res_lst], index=ff.symbols).T
PLUS = pd.DataFrame([res['PLUS'] for res in get_factor1_res_lst], index=ff.symbols).T

Turn_list = Turn.values.tolist()
PLUS_list = PLUS.values.tolist()
get_factor2_para_lst = [(i,np.array(Turn_list[i]),np.array(PLUS_list[i])) for i in range(len(Turn_list))]
with Pool(16) as p:
    get_factor2_res_lst = list(tqdm(p.imap(get_factor2, get_factor2_para_lst), total=len(get_factor2_para_lst))) 
PLUS_deTurn_lst,Turn_dePLUS_lst = zip(*get_factor2_res_lst)
PLUS_deTurn_df = pd.DataFrame(PLUS_deTurn_lst,columns = Turn.columns,index = Turn.index)
Turn_dePLUS_df = pd.DataFrame(Turn_dePLUS_lst,columns = Turn.columns,index = Turn.index)

PLUS_deTurn_df_n = PLUS_deTurn_df.rolling(n2,axis=0,min_periods=1).mean()
Turn_dePLUS_df_n = Turn_dePLUS_df.rolling(n2,axis=0,min_periods=1).mean()
STR_dePLUS = Turn_dePLUS_df.rolling(n2,axis=0,min_periods=1).std()
# 非负处理
PLUS_deTurn_df_n=PLUS_deTurn_df_n.apply(lambda row: row - row.min(), axis=1)
Turn_dePLUS_df_n=Turn_dePLUS_df_n.apply(lambda row: row - row.min(), axis=1)
STR_dePLUS=STR_dePLUS.apply(lambda row: row - row.min(), axis=1)

TPS = Turn_dePLUS_df_n*PLUS_deTurn_df_n
SPS = STR_dePLUS*PLUS_deTurn_df_n

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9181/9181 [00:00<00:00, 15348.20it/s]


In [158]:
TPS.loc[:'2024-03-08 23:59:59',:]

,BTC,ETH,BCH,XRP,TRX,LINK,XLM,ADA,ATOM,BNB,...,MINA,PHB,CFX,ID,ARB,BLUR,SUI,1000PEPE,1000FLOKI,UMA
0,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 04:00:00,0.000000,0.000000,0.000020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 08:00:00,0.000000,0.000000,0.000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 12:00:00,0.000000,0.000000,0.000031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 16:00:00,0.000000,0.000000,0.000014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-08 04:00:00,0.008533,0.012629,0.000000,0.008094,0.006782,0.008279,0.007039,0.009097,0.013328,0.013100,...,0.009782,0.043119,0.010881,0.014097,0.010372,0.008910,0.010961,0.002279,0.001211,0.008634
2024-03-08 08:00:00,0.005719,0.008921,0.000000,0.005491,0.004068,0.005456,0.004850,0.006568,0.009719,0.007997,...,0.006926,0.027236,0.008004,0.010609,0.006563,0.005752,0.007060,0.002255,0.002261,0.005340
2024-03-08 12:00:00,0.003559,0.006539,0.000000,0.004791,0.003009,0.004512,0.004052,0.005676,0.007750,0.006151,...,0.006068,0.027299,0.007139,0.008027,0.005492,0.004574,0.006046,0.002838,0.003137,0.004110


In [104]:
import statsmodels.api as sm
# 纯净化因子
'''
参数解释:
name:货币代码
fre:数据频率
n1:换手率计算参数
n2:turn20计算参数
'''
def get_factor(fre,n1,n2):
    result = {}
    get_factor1_para_lst = [(name,fre,n1) for name in ff.symbols]
    turn_lst = []
    plus_lst = []
    with Pool(16) as p:
        res_lst = list(p.imap(get_factor1, get_factor1_para_lst))  
    Turn = pd.DataFrame([res['Turn'] for res in res_lst], index=ff.symbols).T
    PLUS = pd.DataFrame([res['PLUS'] for res in res_lst], index=ff.symbols).T
    
    Turn_list = Turn.values.tolist()
    PLUS_list = PLUS.values.tolist()
        
    PLUS_deTurn_lst = []
    Turn_dePLUS_lst = []
    for i in range(len(Turn_list)):
        date_Turn = Turn_list[i]
        date_PLUS= PLUS_list[i]
        valid_indices = np.logical_and(~np.isnan(date_Turn), ~np.isnan(date_PLUS))
        date_Turn_valid = date_Turn[valid_indices]
        date_PLUS_valid = date_PLUS[valid_indices]
    
        PLUS_deTurn = np.full_like(date_Turn, np.nan)
        Turn_dePLUS = np.full_like(date_Turn, np.nan)
        # 防止所有位置均为空值的情况
        try:
            date_Turn_valid_c = sm.add_constant(date_Turn_valid)
            date_PLUS_valid_c = sm.add_constant(date_PLUS_valid)
            
            PLUS_deTurn_model = sm.OLS(date_PLUS_valid, date_Turn_valid_c)
            Turn_dePLUS_model = sm.OLS(date_Turn_valid, date_PLUS_valid_c)
            
            PLUS_deTurn_results = PLUS_deTurn_model.fit()
            Turn_dePLUS_results = Turn_dePLUS_model.fit()    
        
            PLUS_deTurn[valid_indices] = PLUS_deTurn_results.resid  
            Turn_dePLUS[valid_indices] = Turn_dePLUS_results.resid  
        except:
            pass    
        PLUS_deTurn_lst.append(PLUS_deTurn)
        Turn_dePLUS_lst.append(Turn_dePLUS)
    PLUS_deTurn_df = pd.DataFrame(PLUS_deTurn_lst,columns = Turn.columns,index = Turn.index)
    Turn_dePLUS_df = pd.DataFrame(Turn_dePLUS_lst,columns = Turn.columns,index = Turn.index)

    PLUS_deTurn_df_n = PLUS_deTurn_df.rolling(n2,axis=0,min_periods=1).mean()
    Turn_dePLUS_df_n = Turn_dePLUS_df.rolling(n2,axis=0,min_periods=1).mean()
    STR_dePLUS = Turn_dePLUS_df.rolling(n2,axis=0,min_periods=1).std()
    # 非负处理
    PLUS_deTurn_df_n=PLUS_deTurn_df_n.apply(lambda row: row - row.min(), axis=1)
    Turn_dePLUS_df_n=Turn_dePLUS_df_n.apply(lambda row: row - row.min(), axis=1)
    STR_dePLUS=STR_dePLUS.apply(lambda row: row - row.min(), axis=1)
    
    TPS = Turn_dePLUS_df_n*PLUS_deTurn_df_n
    SPS = STR_dePLUS*PLUS_deTurn_df_n
    result['TPS'] = TPS
    result['SPS'] = SPS
    return result

In [ ]:
# class factor:
#     def __init__(self,name,fre,n1,n2):
#         self.name = name
#         self.data = ff.read_ba(name,k = f'{fre}H')
#         self.fre = fre
#         self.n1 = n1
#         self.n2 = n2
#     def get_factor(self):
        